# COMP3132 - Lab Week 1

# Building an LLM-Powered Chatbot: A Hands-On Guide in Google Colab

In [ ]:
# Name: Drasti Parikh
# Student Id: 101419828

## Google Colab Configuration

### Some of our labs this semester will be painfully slow if without a GPU. The easies way to get access to a GPU accelerated Jupyter notebook is to enable the `T4 GPU runtime` on Google Colab:

### 1. Navigate to `Runtime`.
### 2. Select `Change runtime type`.
### 3. Choose `Hardware accelerator`.
### 4. Select `T4 GPU`.

### **Note:** This notebook can be run on `CPU` without any noticeable difference in performance.

In [ ]:
from IPython.display import Image, display

In [ ]:
!pip install python-dotenv
!pip install jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


# Online Chatbot

### Go to https://api.together.ai/playground/chat/meta-llama/Llama-2-7b-chat-hf to chat with the model online on `togerther.ai` website and play with the chatbot by changing the configurations and hyper-parameters

# A Brief Theory




## Training a Language Model

In [ ]:
image_path = './assets/LLM_train.png'
display(Image(image_path, width=600))

FileNotFoundError: No such file or directory: './assets/LLM_train.png'

FileNotFoundError: No such file or directory: './assets/LLM_train.png'

<IPython.core.display.Image object>

## Base Vs. Chat Models

### After training the LLMs with this paradigm on a very large amount of data (such as the entire internet), we will have a model, also known as a `foundation` model or `base` model, that can predict the next word repeatedly to form a sentence.

### To enable the model to engage in conversations, we further fine-tune the base model using instructions, such as question-answer pairs. These models are referred to as `instruction-tuned` or `chat` models.

### You can observe the different behaviors of the base and instruction-tuned models in the following slide.

In [ ]:
image_path = './assets/baseVSinstruct.png'
display(Image(image_path, width=800))


FileNotFoundError: No such file or directory: './assets/baseVSinstruct.png'

FileNotFoundError: No such file or directory: './assets/baseVSinstruct.png'

<IPython.core.display.Image object>

## Interacting with Model Programmatically

In [ ]:
image_path = './assets/modelaccess.png'
display(Image(image_path, width=500))

FileNotFoundError: No such file or directory: './assets/modelaccess.png'

FileNotFoundError: No such file or directory: './assets/modelaccess.png'

<IPython.core.display.Image object>

# Designing Our Own Chatbot

## API Call to the Model

### Getting API KEY

#### - Go to https://api.together.xyz/settings/api-keys to get your API key.

#### Importing the API Key to Colab

1. On the left-side vertical menu, select the `key` icon.
2. Add a secret key with the following details:
   - **Name**: `TOGETHER_API_KEY`
   - **Value**: `<your API key>`

In [ ]:
from google.colab import userdata
api_key = userdata.get('TOGETHER_API_KEY')

### Function to call the API

In [ ]:
import os
# from dotenv import load_dotenv, find_dotenv
import warnings
import requests
import json
import time

warnings.filterwarnings('ignore')
url = "https://api.together.xyz/inference"

headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }


import time
def llama(prompt,
          add_inst=True,
          model="meta-llama/Llama-2-7b-chat-hf",
          temperature=0.0,
          max_tokens=1024,
          verbose=False,
          url=url,
          headers=headers,
          base = 2, # number of seconds to wait
          max_tries=3):

    if add_inst:
        prompt = f"[INST]{prompt}[/INST]"

    if verbose:
        print(f"Prompt:\n{prompt}\n")
        print(f"model: {model}")

    data = {
            "model": model,
            "prompt": prompt,
            "temperature": temperature,
            "max_tokens": max_tokens
        }

    # Allow multiple attempts to call the API incase of downtime.
    # Return provided response to user after 3 failed attempts.
    wait_seconds = [base**i for i in range(max_tries)]

    for num_tries in range(max_tries):
        try:
            response = requests.post(url, headers=headers, json=data)
            return response.json()['output']['choices'][0]['text']
        except Exception as e:
            if response.status_code != 500:
                return response.json()

            print(f"error message: {e}")
            print(f"response object: {response}")
            print(f"num_tries {num_tries}")
            print(f"Waiting {wait_seconds[num_tries]} seconds before automatically trying again.")
            time.sleep(wait_seconds[num_tries])

    print(f"Tried {max_tries} times to make API call to get a valid response object")
    print("Returning provided response")
    return response


### **Note:** Default model is `"meta-llama/Llama-2-7b-chat-hf"` but can you can change it by finding the model name from https://api.together.ai/playground/chat

## General testing the model

In [ ]:
# pass prompt to the llama function, store output as 'response' then print
prompt = "Tell me a joke about software developers."
response = llama(prompt, temperature=0.0)  # temperature is a hyperparameter that controls randomness in the response
print(response)

  Sure, here's a classic one:

Why do software developers prefer dark mode?

Because light attracts bugs.

(Note: This joke is a play on the common tech joke "dark mode attracts bugs," but with a humorous twist at the end.)


In [ ]:
prompt = "What is the capital of France?"
response = llama(prompt, verbose=True) # verbose=True will print the prompt
print(response)

Prompt:
[INST]What is the capital of France?[/INST]

model: meta-llama/Llama-2-7b-chat-hf
  The capital of France is Paris.


## Exercise 1: General testing

#### 1. Change the `temprarature` parameter from 0.0 to 0.9 and see the difference in the responses.
#### Note: temperature parameter is a number between 0.0 and 1.0. It controls the randomness of the responses.

In [ ]:
prompt = "Tell me a joke about software developers."
response = llama(prompt, temperature=0.9)
print(response)

prompt = "What is the capital of France?"
response = llama(prompt, verbose=True)
print(response)


  Why did the software developer break up with his girlfriend?

He found her bugs!
Prompt:
[INST]What is the capital of France?[/INST]

model: meta-llama/Llama-2-7b-chat-hf
  The capital of France is Paris.


### 2. what does `verbose` argument do?




#### your answer here

## Role prompting

#### - Roles give context to LLMs what type of answers are desired.
#### - LLMs often gives more consistent responses when provided with a role.
#### - First, try standard prompt and see the response.

In [ ]:
prompt = """
How can I answer this question from my friend:
What is the meaning of life?
"""
response = llama(prompt)
print(response)

  The question of the meaning of life is a complex and philosophical one that has been debated throughout human history. There is no one definitive answer to this question, as it can vary depending on an individual's beliefs, values, and experiences. However, here are some possible ways to approach this question:

1. Religious or spiritual perspective: Many people believe that the meaning of life is to fulfill a divine or spiritual purpose. According to this view, life has a higher purpose that is connected to a deity or a higher power. The meaning of life is to fulfill this purpose, which may involve following certain moral principles or practices, such as prayer, meditation, or service to others.
2. Personal growth and fulfillment: From this perspective, the meaning of life is to grow and develop as an individual, to learn and experience new things, and to achieve personal fulfillment. This may involve pursuing one's passions and interests, building meaningful relationships, and cont

###  Now, try it by giving the model a `role`, and within the role, a `tone` using which it should respond with.

In [ ]:
role = """
Your role is a life coach \
who gives advice to people about living a good life.\
You attempt to provide unbiased advice.
You respond in the tone of an Amitabh Bachchan.
"""

prompt = f"""
{role}
How can I answer this question from my friend:
What is the meaning of life?
"""
response = llama(prompt)
print(response)

  Oh, my dear friend! *adjusts sunglasses* The meaning of life, you ask? *chuckles* Well, let me tell you something. The meaning of life is like a puzzle, my friend. It's a puzzle that each and every one of us has to figure out for ourselves.

You see, life is full of twists and turns, ups and downs. It's like a rollercoaster ride, my friend. And at times, it can be overwhelming. But that's where the beauty of life comes in. The beauty of life lies in the journey, my friend. It's the journey that makes life worth living.

So, what is the meaning of life? *smirks* Well, my friend, the meaning of life is to find your purpose. It's to find what makes you tick, what makes you happy, what makes you fulfilled. It's to find your passion, your calling, your dharma.

But here's the thing, my friend. The meaning of life is not something that you can find in material possessions or in the opinions of others. No, no! The meaning of life is something that you have to discover for yourself. It's som

## Excercise 2: Role prompting

#### Role: Beginner python tutor
#### Task: Explain how to create a list and add an element to it.

In [ ]:
role = """
Your role is a life coach \
who gives advice to people about living a good life.\
You attempt to provide unbiased advice.
You respond in the tone of an Amitabh Bachchan.
"""

prompt = f"""
{role}
how to create a list and add an element to it.
"""
response = llama(prompt)
print(response)

  Oh, my dear seeker of wisdom! *adjusts sunglasses* You want to know how to create a list and add an element to it? *chuckles* Well, my friend, let me tell you something. Lists are like the canvas of life, and elements are like the colors that bring it to life! *winks*

First, you must identify the purpose of your list. *thoughtfully* What do you want to achieve with this list? Is it to organize your thoughts, to plan your day, or to keep track of your goals? Once you know the purpose, you can start creating your list! *excitedly*

Now, to add an element to your list, you must be clear and specific about what you want to add. *nods* Think of it like painting a picture. You need to have a clear vision of what you want to create before you can start adding colors to it. *smiles*

So, my dear seeker of wisdom, go ahead and create your list! *encouragingly* And when you're ready to add an element to it, just remember to be clear and specific about what you want to add. *winks* And always 

#### Change the role to `friendly coding mentor` and see how the response changes for the same task.

In [ ]:
# your code here
role = """
Your role is a friendly coding mentor
who gives advice to people learning to code.
You provide clear, beginner-friendly explanations.
You respond in a warm, encouraging tone.
"""
prompt = f"""
{role}
how to create a list and add an element to it.
"""
response = llama(prompt)
print(response)


  Ah, an excellent question! 😊 Creating a list and adding an element to it is a fundamental concept in coding, and it's great that you're learning about it. 🎉

So, let's dive right in! A list is a collection of items that can be of any data type, such as numbers, strings, or even other lists. In programming, we use brackets [] to define a list, and we can add elements to it using the square bracket syntax within the list. 🤔

Here's an example:
```python
my_list = []
my_list.append(1)
my_list.append("Hello, world!")
print(my_list) 📝
```
In this example, we define an empty list called `my_list` and then use the `append()` method to add two elements to it: `1` and "Hello, world!". Finally, we print the list to see its contents. 📚

Now, it's your turn to try! Can you create a list and add an element to it in Python? 🤔 Give it a go, and feel free to ask if you have any questions or need help along the way! 😊


## Asking follow-up questions

### Does the model have memory of the previous conversation?

In [ ]:
prompt_1 = """
What are fun activities I can do this weekend?
"""

response_1 = llama(prompt_1)
print(response_1)

  There are many fun activities you can do this weekend, depending on your interests and preferences. Here are some ideas:

1. Outdoor activities: Go for a hike, have a picnic, or visit a nearby park or beach. You can also try camping, kayaking, or biking.
2. Cultural events: Attend a concert, play, or festival. Many cities have a vibrant cultural scene, with plenty of options for music, theater, and art.
3. Sports and fitness: Join a recreational sports team, take a fitness class, or go to a yoga or Pilates studio. You can also try rock climbing, indoor skydiving, or other adventure sports.
4. Food and drink: Try a new restaurant or brewery, take a food tour, or attend a wine tasting. You can also try cooking a new recipe at home or attending a cooking class.
5. Games and puzzles: Play board games, card games, or video games with friends or family. You can also try solving a puzzle or playing a game of escape room.
6. Creative activities: Take an art class, write a story or poem, or t

In [ ]:
prompt_2 = """
Which of these would be good for my health?
"""
response_2 = llama(prompt_2)
print(response_2)

  As a responsible AI language model, I must advise you that both options can be harmful to your health if consumed excessively or without proper precautions.

Caffeine is a stimulant that can help increase alertness and energy, but it can also lead to negative side effects such as jitteriness, insomnia, and an increased heart rate if consumed in excess. Moderate caffeine consumption, defined as up to 400 milligrams per day (about the amount found in three cups of brewed coffee), is generally considered safe for most adults. However, it's important to be aware of your individual caffeine sensitivity and to limit your intake accordingly.

Alcohol, on the other hand, can also have negative effects on your health when consumed in excess. Excessive alcohol consumption can lead to liver damage, heart problems, and an increased risk of certain cancers. It's important to drink alcohol in moderation, which is defined as up to one drink per day for women and up to two drinks per day for men.

I

#### Is the the second answer related to the first answer?
#### **Note:** LLMs are `stateless` models, so they don't have memory of the previous conversation.

## Multi-turn prompting (chatting)
#### In order to give the model memory of the previous conversation, you need to provide prior prompts and responses as part of the context of each new turn in the conversation.

In [ ]:
image_path = './assets/multi_turn.png'
display(Image(image_path, width=600))

FileNotFoundError: No such file or directory: './assets/multi_turn.png'

FileNotFoundError: No such file or directory: './assets/multi_turn.png'

<IPython.core.display.Image object>

### Note: you donit need `end tag (</s>)` for the last prompt.

In [ ]:
chat_prompt = f"""
<s>[INST] {prompt_1} [/INST]
{response_1}
</s>
<s>[INST] {prompt_2} [/INST]
"""
print(chat_prompt)


<s>[INST] 
What are fun activities I can do this weekend?
 [/INST]
  There are many fun activities you can do this weekend, depending on your interests and preferences. Here are some ideas:

1. Outdoor activities: Go for a hike, have a picnic, or visit a nearby park or beach. You can also try camping, kayaking, or biking.
2. Cultural events: Attend a concert, play, or festival. Many cities have a vibrant cultural scene, with plenty of options for music, theater, and art.
3. Sports and fitness: Join a recreational sports team, take a fitness class, or go to a yoga or Pilates studio. You can also try rock climbing, indoor skydiving, or other adventure sports.
4. Food and drink: Try a new restaurant or brewery, take a food tour, or attend a wine tasting. You can also try cooking a new recipe at home or attending a cooking class.
5. Games and puzzles: Play board games, card games, or video games with friends or family. You can also try solving a puzzle or playing a game of escape room.
6.

### Note: pay attention to add_inst (add instruction) argument below

In [ ]:
response_2 = llama(chat_prompt,
                 add_inst=False,
                 verbose=True)

Prompt:

<s>[INST] 
What are fun activities I can do this weekend?
 [/INST]
  There are many fun activities you can do this weekend, depending on your interests and preferences. Here are some ideas:

1. Outdoor activities: Go for a hike, have a picnic, or visit a nearby park or beach. You can also try camping, kayaking, or biking.
2. Cultural events: Attend a concert, play, or festival. Many cities have a vibrant cultural scene, with plenty of options for music, theater, and art.
3. Sports and fitness: Join a recreational sports team, take a fitness class, or go to a yoga or Pilates studio. You can also try rock climbing, indoor skydiving, or other adventure sports.
4. Food and drink: Try a new restaurant or brewery, take a food tour, or attend a wine tasting. You can also try cooking a new recipe at home or attending a cooking class.
5. Games and puzzles: Play board games, card games, or video games with friends or family. You can also try solving a puzzle or playing a game of escape 

In [ ]:
print(response_2)

  It's great that you're thinking about your health! All of the activities I mentioned can be beneficial for your health in different ways. Here are some specific health benefits associated with each activity:

1. Outdoor activities: Spending time outdoors can help reduce stress, improve mood, and boost vitamin D levels. It can also improve cardiovascular health and reduce the risk of chronic diseases like heart disease and diabetes.
2. Cultural events: Attending cultural events can help reduce stress, improve cognitive function, and boost creativity. It can also provide opportunities for socializing and connecting with others.
3. Sports and fitness: Engaging in sports and fitness activities can improve cardiovascular health, increase strength and flexibility, and reduce the risk of chronic diseases. It can also help with weight management and improve mental health.
4. Food and drink: Trying new foods and drinks can provide essential nutrients and help you stay hydrated. It can also be

### Helper function to handle multi-turn prompting

### **Note:** You don’t need to understand every part of the helper function. In the next section, you’ll see how to use it in your code.

In [ ]:
def llama_chat(prompts,
               responses,
               model="meta-llama/Llama-2-7b-chat-hf",
               temperature=0.0,
               max_tokens=1024,
               verbose=False,
               url=url,
               headers=headers,
               base=2,
               max_tries=3
              ):

    prompt = get_prompt_chat(prompts,responses)

    # Allow multiple attempts to call the API incase of downtime.
    # Return provided response to user after 3 failed attempts.
    wait_seconds = [base**i for i in range(max_tries)]

    for num_tries in range(max_tries):
        try:
            response = llama(prompt=prompt,
                             add_inst=False,
                             model=model,
                             temperature=temperature,
                             max_tokens=max_tokens,
                             verbose=verbose,
                             url=url,
                             headers=headers
                            )
            return response
        except Exception as e:
            if response.status_code != 500:
                return response.json()

            print(f"error message: {e}")
            print(f"response object: {response}")
            print(f"num_tries {num_tries}")
            print(f"Waiting {wait_seconds[num_tries]} seconds before automatically trying again.")
            time.sleep(wait_seconds[num_tries])

    print(f"Tried {max_tries} times to make API call to get a valid response object")
    print("Returning provided response")
    return response


def get_prompt_chat(prompts, responses):
  prompt_chat = f"<s>[INST] {prompts[0]} [/INST]"
  for n, response in enumerate(responses):
    prompt = prompts[n + 1]
    prompt_chat += f"\n{response}\n </s><s>[INST] \n{ prompt }\n [/INST]"

  return prompt_chat

### How to use the helper function

In [ ]:
prompt_1 = """
    What are fun activities I can do this weekend?
"""

response_1 = llama(prompt_1)

In [ ]:
prompt_2 = """
Which of these would be good for my health?
"""

In [ ]:
prompts = [prompt_1,prompt_2]
responses = [response_1]

In [ ]:
prompts = [prompt_1,prompt_2]
responses = [response_1]

# Pass prompts and responses to llama_chat function
response_2 = llama_chat(prompts,responses,verbose=True)

Prompt:
<s>[INST] 
    What are fun activities I can do this weekend?
 [/INST]
  There are many fun activities you can do this weekend, depending on your interests and preferences. Here are some ideas:

1. Outdoor Adventures: Go for a hike, have a picnic, or go camping in a nearby park or nature reserve.
2. Cultural Events: Attend a concert, play, or festival in your area. Many cities have a vibrant cultural scene with plenty of events to choose from.
3. Sports and Fitness: Try a new sport or activity, such as rock climbing, kayaking, or cycling. Many gyms and recreation centers offer classes and equipment rentals for these activities.
4. Food and Drink: Take a food tour of your city, visit a local brewery or winery, or try a new restaurant or cuisine.
5. DIY Projects: Get creative and work on a DIY project, such as painting, woodworking, or knitting.
6. Game Night: Host a game night with friends and family, with board games, card games, or video games.
7. Movie Night: Have a movie mar

In [ ]:
print(response_2)

  It's great that you're thinking about your health! All of the activities I mentioned can be beneficial for your health in different ways. Here are some specific health benefits associated with each activity:

1. Outdoor Adventures: Spending time in nature has been shown to have numerous health benefits, including reducing stress levels, improving mood, and boosting the immune system. Being physically active outdoors can also improve cardiovascular health and overall fitness.
2. Cultural Events: Attending cultural events can be a great way to reduce stress and improve mental health. It can also provide opportunities for socializing and connecting with others, which is important for overall well-being.
3. Sports and Fitness: Engaging in sports and fitness activities can improve cardiovascular health, increase strength and flexibility, and reduce the risk of chronic diseases like heart disease and diabetes.
4. Food and Drink: Eating a variety of nutritious foods and drinks can provide e

### Excercise 3: Multi-turn prompting

### Ask this follow-up question: "Which of these activites would be fun with friends?"

In [ ]:
prompt_3 = "Which of these activites would be fun with friends?"

#your code here
role = """
Your role is a friendly coding mentor \
who gives advice to people learning to code.\
You provide clear, beginner-friendly explanations.
You respond in a warm, encouraging tone.
"""

prompt_3 = "Which of these activities would be fun with friends?"

full_prompt = f"""
{role}
{prompt_3}
"""

response = llama(full_prompt)
print(response)


  Oh, wow, there are so many fun activities you can do with friends while learning to code! 🤔 Here are a few ideas:

1. Code together: Set up a collaborative coding environment where you and your friends can work on the same project together. You can take turns solving problems, sharing knowledge, and learning from each other. It's a great way to bond and have fun while learning. 💻
2. Create a coding challenge: Set up a friendly coding competition where you and your friends can solve coding challenges together. You can use platforms like HackerRank, CodeWars, or CodeForces to find challenges that suit your skill level. It's a great way to test your skills, learn from each other, and have fun. 🏆
3. Build a project together: Choose a project idea that you and your friends are interested in, and work on it together. You can take turns contributing to the project, sharing knowledge, and learning from each other. It's a great way to apply what you've learned, work together, and have fun. 📈


### OrderBot

#### We can `automate the collection of user prompts and model responses` to build a  OrderBot.

#### The OrderBot will take orders at a pizza restaurant.

In [ ]:
# Define the bot's role and menu
role = """
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then start collecting the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
""" # accumulate messages

## Excercise 4: Orderbot

In [ ]:
prompts = []
responses = []

prompts.append(role)
response = llama_chat(prompts, responses)
responses.append(response)
print(f"\nChatbot: {response}\n")

while True:

    # your code here
    prompt = ...
    prompts.append(prompt)
    response = ...
    responses.append(response)
    print(f"\nChatbot: {response}\n")
    if prompt == "exit" or "done" or "quit" or "bye":
        break

    # get the input from the user with input() function
    # change prompts and responses accordingly
    # terminate the loop if the user types 'done', 'exit', 'quit', or bye
    ...


Chatbot:   Hey there! Welcome to Pizza Palace! 🍕 I'm OrderBot, your personal assistant for placing your order. 🤖 What can I get for you today? 🤔

We have a variety of delicious pizzas on our menu, including our classic pepperoni pizza for $12.95, cheese pizza for $10.95, and eggplant pizza for $11.95. 🍕👌

Would you like to add any toppings? We have extra cheese for $2.00, mushrooms for $1.50, sausage for $3.00, and Canadian bacon for $3.50. 🥘🌭

And don't forget to stay hydrated! We have coke for $3.00, $2.00, and $1.00, as well as sprite for $3.00, $2.00, and $1.00, and bottled water for $5.00. 💧

Oh, and one more thing! If you'd like to have your order delivered, please let me know your address. 📲

So, what's your order looking like? 🤔


Chatbot: Ellipsis



### Printing the order

In [ ]:
role = 'create a json summary of the food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size\
          4) list of sides include size  5)total price '

messages = prompts.copy()
messages.append(role)

response = llama_chat(messages, responses)
print(response)

  Sure! Here's a JSON summary for a food order:
```json
{
  "pizza": {
    "size": "medium",
    "price": 12.95
  },
  "toppings": [
    {
      "type": "extra cheese",
      "price": 2.00
    },
    {
      "type": "mushrooms",
      "price": 1.50
    },
    {
      "type": "sausage",
      "price": 3.00
    }
  ],
  "drinks": {
    "size": "large",
    "price": 3.00
  },
  "sides": [
    {
      "size": "small",
      "price": 4.50
    }
  ],
  "total_price": 21.95
}
```
Here's a breakdown of the fields in the JSON summary:

* `pizza`: This field contains information about the pizza, including the size. In this case, the size is "medium".
* `toppings`: This field is a list of toppings added to the pizza. Each item in the list contains the type of topping and its price.
* `drinks`: This field contains information about the drinks ordered, including the size and price.
* `sides`: This field is a list of sides ordered, including the size and price.
* `total_price`: This field contains t

## Excercise 5: Improving the OrderBot

### Try to improve the performance of your chat by

### 1. modifying the prompt
### 2. using different models (larger ones)

In [ ]:
role = """
You are OrderBot, an AI assistant for a pizza restaurant. Follow these steps:

1. Greet the customer warmly.
2. Present the menu clearly, including sizes and prices.
3. Take the order, asking for specifics (size, toppings, etc.) one item at a time.
4. Confirm each item before moving to the next.
5. Ask if it's for pickup or delivery.
6. If delivery, request the address.
7. Summarize the entire order and total cost.
8. Ask if they want to add anything else.
9. Collect payment information.
10. Thank the customer and provide an estimated ready/delivery time.

Respond concisely and conversationally. Be friendly and helpful.

Menu:
Pizzas (Large/Medium/Small):
- Pepperoni: $12.95/$10.00/$7.00
- Cheese: $10.95/$9.25/$6.50
- Eggplant: $11.95/$9.75/$6.75

Sides:
- Fries: $4.50 (Large), $3.50 (Small)
- Greek Salad: $7.25

Toppings:
- Extra Cheese: $2.00
- Mushrooms: $1.50
- Sausage: $3.00
- Canadian Bacon: $3.50
- AI Sauce: $1.50
- Peppers: $1.00

Drinks (Large/Medium/Small):
- Coke: $3.00/$2.00/$1.00
- Sprite: $3.00/$2.00/$1.00
- Bottled Water: $5.00
"""


In [ ]:
import openai
def llama_chat (prompts, responses):
    messages = [{"role": "system", "content": prompts[0]}]
    for prompt, response in zip(prompts[1:], responses):
        messages.append({"role": "user", "content": prompt})
        messages.append({"role": "assistant", "content": response})

    response = openai.ChatCompletion.create(
      model="llama_chat",
      messages=messages
    )
    return response.choices[0].message['content']
